In [1]:
import torch
from keras.preprocessing.sequence import pad_sequences
from pytorch_pretrained_bert import BertTokenizer
from pytorch_pretrained_bert import BertForSequenceClassification
import pandas as pd
import numpy as np
import gc

Using TensorFlow backend.


In [2]:
device = "cpu"
if torch.cuda.is_available():
    torch.cuda.empty_cache()
gc.collect()
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)
#model = torch.load("models/bert-based-uncased.pth")
model.load_state_dict(torch.load('models/bert-based-uncased-GED.pth'))
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased',do_lower_case = True)
#tokenizer = torch.load('models/bert-based-uncased.pth')

In [3]:
## input sentences
#sentences = ["The sun is shiny today as well as the roses", "I drinked water", "I will be there at time"]
sentences = ["Hi, my name is Akshay. I am currently pursing masters from IIITB. My favourite subjects are Electronics and Computer Science"]

In [4]:
model.eval()
sentences = ["[CLS] " + sentence + " [SEP]" for sentence in sentences]
labels =[0]

tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]
MAX_LEN = 128
predictions = []
true_labels = []
input_ids = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts],maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN,dtype ="long", truncating="post",padding ="post")

attention_masks = []
for seq in input_ids:
    seq_mask = [float(i > 0) for i in seq]
    attention_masks.append(seq_mask)

prediction_inputs = torch.tensor(input_ids)
prediction_masks = torch.tensor(attention_masks)
prediction_labels = torch.tensor(labels)
model.eval()

with torch.no_grad():
    logits = model(prediction_inputs.to(device).long(), token_type_ids=None, attention_mask=prediction_masks.to(device).long())
logits = logits.detach().cpu().numpy()
predictions.append(logits)

flat_predictions = [item for sublist in predictions for item in sublist]
flat_predictions = np.argmax(flat_predictions, axis=1).flatten()
flat_true_labels = [item for sublist in true_labels for item in sublist]

In [5]:
flat_predictions

array([1], dtype=int64)